In [41]:
import geopandas as gpd
from shapely.geometry import Point
import folium
from IPython.display import IFrame
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import io

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


# 데이터 프레임 생성
data = pd.DataFrame({
    'City': ['200. 국회의원회관', '201. 진미파라곤 앞', '202. 국민일보 앞', '203. 국회의사당역 3번출구 옆', '204. 국회의사당역 5번출구 옆', '205. 산업은행 앞', 
             '206. KBS 앞', '207. 여의나루역 1번출구 앞', '209. 유진투자증권빌딩 앞', '213. KT앞', '214. 금융감독원 앞', '215. 여의도고교 앞', '217. NH농협은행 앞', 
             '219. 롯데캐슬엠파이어 옆', '220. 미성아파트 A동 앞', '221. 여의도초교 앞', '222. 시범아파트버스정류장 옆', '223. 진주아파트상가 앞', '224. 롯데캐슬 앞', 
             '225. 앙카라공원 앞', '226. 샛강역 1번출구 앞', '248. 초원아파트 앞', '249. 여의도중학교 옆', '260. 여의도 마리나선착장 앞', '270. 증권거래소후문교차로', 
             '271. 샛강생태공원방문자센터 앞', '297.국회3문', '298.KBS 연구동 건너편', '299.여의도 순복음교회', '4563. 여의도 은하아파트', '4564. 63스퀘어',
             '4579. 국회의사당 본관', '4580. 국회3문', '4581. 국회소통관', '4582. 국회도서관', '4583. 의정관', '4584. 국회2문', '4585. 국회6문', '4589. KRX한국거래소(1)',
             '5853. 여의도역2번출구 앞', '5854. 신한금융투자 앞', '5855. 하이투자증권 앞', '5859. 여의도 삼부아파트'],
    
    'Latitude': [
                37.52841568, 37.53123856, 37.52881622, 37.5280571, 37.52816391, 37.52626419, 37.52466583, 37.52715683, 
                37.52461243, 37.52190781, 37.5230217, 37.52483749, 37.52207947, 37.52069473, 37.5193634, 37.52267456, 
                37.51943588, 37.51913071, 37.5200882, 37.51758575, 37.51776505, 37.53105545, 37.52434921, 37.53365707, 
                37.52234268, 37.51896286, 37.52863693, 37.52750778,37.53179169, 37.51850891, 37.52070999, 37.5325737, 
                37.52974701, 37.53050232, 37.53054047, 37.53214645,37.52864075, 37.53271866, 37.52347946, 37.52260971, 
                37.52508926, 37.52464294, 37.52539444],
    
    'Longitude': [
                126.9139175, 126.9213333, 126.92453, 126.9187012, 126.9170227, 126.9205093, 126.9180222, 126.9319, 
                126.9278336, 126.9189529, 126.9208374, 126.934906, 126.9303665, 126.9258347, 126.9260483, 126.9377899, 
                126.9375992, 126.9333725, 126.9323654, 126.9296188, 126.9284134, 126.9242096, 126.9361191, 126.9121628, 126.9271011, 126.9216156, 126.912674, 126.914299, 
                126.9224548, 126.9344864, 126.9396591, 126.9140472, 126.9131699, 126.9128952, 126.9181747, 
                126.9174728, 126.9151077, 126.9179916, 126.9271164, 126.9230118, 126.9240799, 126.9234238, 126.9304581]

})

# 데이터 프레임을 GeoDataFrame으로 변환
geometry = [Point(xy) for xy in zip(data['Longitude'], data['Latitude'])]
gdf = gpd.GeoDataFrame(data, geometry=geometry)

# 여의도의 중심 좌표를 사용하여 지도 초기화
center = [37.52158, 126.9243]

# Additional data
additional_data = pd.DataFrame({
    'City': ['여의도', '여의나루', '국회의사당', '샛강'],
    'Latitude': [37.521578, 37.527145, 37.528105, 37.517274],
    'Longitude': [126.924318, 126.932807, 126.917874, 126.928422]
})

# New data
new_data = pd.DataFrame({
    'City': ['국민일보 앞', '여의나루역 1번출구 앞', '신한금융투자 앞'],
    'Latitude': [37.52881622, 37.52715683, 37.52508926],
    'Longitude': [126.92453, 126.9319, 126.9240799]
})

# Combine all the data
combined_data = pd.concat([data, additional_data, new_data], ignore_index=True)

# Convert the combined data to GeoDataFrame
combined_geometry = [Point(xy) for xy in zip(combined_data['Longitude'], combined_data['Latitude'])]
combined_gdf = gpd.GeoDataFrame(combined_data, geometry=combined_geometry)

# Update the map with the new GeoDataFrame
m = folium.Map(location=center, zoom_start=15)

# Add markers for all locations
for idx, row in combined_gdf.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['City']).add_to(m)

# Add red buffer for each location in the original data
buffer_distance_meters_red = 0.0015  # Set the buffer distance to 0.0015 meters (red)
gdf['Buffer'] = gdf['geometry'].buffer(buffer_distance_meters_red)

for idx, row in gdf.iterrows():
    folium.GeoJson(row['Buffer'].__geo_interface__,
                   style_function=lambda x: {'fillColor': 'red'}).add_to(m)

# Add blue buffer for each location in the new and additional data
buffer_distance_meters_blue = 300  # Set the buffer distance to 300 meters (blue)
for idx, row in pd.concat([additional_data, new_data], ignore_index=True).iterrows():
    folium.Circle([row['Latitude'], row['Longitude']], radius=buffer_distance_meters_blue, color='blue', fill=True).add_to(m)

# Add green buffer for each location in the new data
buffer_distance_meters_green = 0.002  # Set the buffer distance to 0.002 meters (green)
for idx, row in new_data.iterrows():
    point = Point(row['Longitude'], row['Latitude'])
    buffer_geometry = point.buffer(buffer_distance_meters_green)
    folium.GeoJson(buffer_geometry.__geo_interface__,
                   style_function=lambda x: {'fillColor': 'green'}).add_to(m)

# Save the map as an HTML file
html_file_path = 'map_with_red_blue_green_buffers.html'
m.save(html_file_path)

# Dash 앱 초기화
app = dash.Dash(__name__)

# Dash 레이아웃 구성
app.layout = html.Div([
    html.Iframe(srcDoc=io.open(html_file_path, 'r', encoding='utf-8').read(), width='100%', height='500')
])

# 앱 실행
if __name__ == '__main__':
    app.run_server(debug=True, port=1014)

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import folium
from IPython.display import IFrame
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import io

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


# 데이터 프레임 생성
data = pd.DataFrame({
    'City': ['200. 국회의원회관', '201. 진미파라곤 앞', '202. 국민일보 앞', '203. 국회의사당역 3번출구 옆', '204. 국회의사당역 5번출구 옆', '205. 산업은행 앞', 
             '206. KBS 앞', '207. 여의나루역 1번출구 앞', '209. 유진투자증권빌딩 앞', '213. KT앞', '214. 금융감독원 앞', '215. 여의도고교 앞', '217. NH농협은행 앞', 
             '219. 롯데캐슬엠파이어 옆', '220. 미성아파트 A동 앞', '221. 여의도초교 앞', '222. 시범아파트버스정류장 옆', '223. 진주아파트상가 앞', '224. 롯데캐슬 앞', 
             '225. 앙카라공원 앞', '226. 샛강역 1번출구 앞', '248. 초원아파트 앞', '249. 여의도중학교 옆', '260. 여의도 마리나선착장 앞', '270. 증권거래소후문교차로', 
             '271. 샛강생태공원방문자센터 앞', '297.국회3문', '298.KBS 연구동 건너편', '299.여의도 순복음교회', '4563. 여의도 은하아파트', '4564. 63스퀘어',
             '4579. 국회의사당 본관', '4580. 국회3문', '4581. 국회소통관', '4582. 국회도서관', '4583. 의정관', '4584. 국회2문', '4585. 국회6문', '4589. KRX한국거래소(1)',
             '5853. 여의도역2번출구 앞', '5854. 신한금융투자 앞', '5855. 하이투자증권 앞', '5859. 여의도 삼부아파트'],
    
    'Latitude': [
                37.52841568, 37.53123856, 37.52881622, 37.5280571, 37.52816391, 37.52626419, 37.52466583, 37.52715683, 
                37.52461243, 37.52190781, 37.5230217, 37.52483749, 37.52207947, 37.52069473, 37.5193634, 37.52267456, 
                37.51943588, 37.51913071, 37.5200882, 37.51758575, 37.51776505, 37.53105545, 37.52434921, 37.53365707, 
                37.52234268, 37.51896286, 37.52863693, 37.52750778,37.53179169, 37.51850891, 37.52070999, 37.5325737, 
                37.52974701, 37.53050232, 37.53054047, 37.53214645,37.52864075, 37.53271866, 37.52347946, 37.52260971, 
                37.52508926, 37.52464294, 37.52539444],
    
    'Longitude': [
                126.9139175, 126.9213333, 126.92453, 126.9187012, 126.9170227, 126.9205093, 126.9180222, 126.9319, 
                126.9278336, 126.9189529, 126.9208374, 126.934906, 126.9303665, 126.9258347, 126.9260483, 126.9377899, 
                126.9375992, 126.9333725, 126.9323654, 126.9296188, 126.9284134, 126.9242096, 126.9361191, 126.9121628, 126.9271011, 126.9216156, 126.912674, 126.914299, 
                126.9224548, 126.9344864, 126.9396591, 126.9140472, 126.9131699, 126.9128952, 126.9181747, 
                126.9174728, 126.9151077, 126.9179916, 126.9271164, 126.9230118, 126.9240799, 126.9234238, 126.9304581]

})

# 데이터 프레임을 GeoDataFrame으로 변환
geometry = [Point(xy) for xy in zip(data['Longitude'], data['Latitude'])]
gdf = gpd.GeoDataFrame(data, geometry=geometry)

# 여의도의 중심 좌표를 사용하여 지도 초기화
center = [37.52158, 126.9243]

# Additional data
additional_data = pd.DataFrame({
    'City': ['여의도', '여의나루', '국회의사당', '샛강'],
    'Latitude': [37.521578, 37.527145, 37.528105, 37.517274],
    'Longitude': [126.924318, 126.932807, 126.917874, 126.928422]
})

# New data
new_data = pd.DataFrame({
    'City': ['국민일보 앞', '여의나루역 1번출구 앞', '신한금융투자 앞'],
    'Latitude': [37.52881622, 37.52715683, 37.52508926],
    'Longitude': [126.92453, 126.9319, 126.9240799]
})

# Combine all the data
combined_data = pd.concat([data, additional_data, new_data], ignore_index=True)

# Convert the combined data to GeoDataFrame
combined_geometry = [Point(xy) for xy in zip(combined_data['Longitude'], combined_data['Latitude'])]
combined_gdf = gpd.GeoDataFrame(combined_data, geometry=combined_geometry)

# Update the map with the new GeoDataFrame
m = folium.Map(location=center, zoom_start=15)

# Add markers for all locations
for idx, row in combined_gdf.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['City']).add_to(m)

# Add red buffer for each location in the original data
buffer_distance_meters_red = 0.0015  # Set the buffer distance to 0.0015 meters (red)
gdf['Buffer'] = gdf['geometry'].buffer(buffer_distance_meters_red)

for idx, row in gdf.iterrows():
    folium.GeoJson(row['Buffer'].__geo_interface__,
                   style_function=lambda x: {'fillColor': 'red'}).add_to(m)

# Add blue buffer for each location in the new and additional data
buffer_distance_meters_blue = 300  # Set the buffer distance to 300 meters (blue)
for idx, row in pd.concat([additional_data, new_data], ignore_index=True).iterrows():
    folium.Circle([row['Latitude'], row['Longitude']], radius=buffer_distance_meters_blue, color='blue', fill=True).add_to(m)

# Add green buffer for each location in the new data
buffer_distance_meters_green = 0.002  # Set the buffer distance to 0.002 meters (green)
for idx, row in new_data.iterrows():
    point = Point(row['Longitude'], row['Latitude'])
    buffer_geometry = point.buffer(buffer_distance_meters_green)
    folium.GeoJson(buffer_geometry.__geo_interface__,
                   style_function=lambda x: {'fillColor': 'green'}).add_to(m)

# Save the map as an HTML file
html_file_path = 'map_with_red_blue_green_buffers.html'
m.save(html_file_path)

# Dash 앱 초기화
app = dash.Dash(__name__)

# Dash 레이아웃 구성
app.layout = html.Div([
    html.Iframe(srcDoc=io.open(html_file_path, 'r', encoding='utf-8').read(), width='100%', height='500')
])

# 앱 실행
if __name__ == '__main__':
    app.run_server(debug=True, port=1014)

In [46]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from IPython.display import IFrame
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import io

import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd


    
data = pd.DataFrame({
    'City': ['200. 국회의원회관', '201. 진미파라곤 앞', '202. 국민일보 앞', '203. 국회의사당역 3번출구 옆', '204. 국회의사당역 5번출구 옆', '205. 산업은행 앞', 
             '206. KBS 앞', '207. 여의나루역 1번출구 앞', '209. 유진투자증권빌딩 앞', '213. KT앞', '214. 금융감독원 앞', '215. 여의도고교 앞', '217. NH농협은행 앞', 
             '219. 롯데캐슬엠파이어 옆', '220. 미성아파트 A동 앞', '221. 여의도초교 앞', '222. 시범아파트버스정류장 옆', '223. 진주아파트상가 앞', '224. 롯데캐슬 앞', 
             '225. 앙카라공원 앞', '226. 샛강역 1번출구 앞', '248. 초원아파트 앞', '249. 여의도중학교 옆', '260. 여의도 마리나선착장 앞', '270. 증권거래소후문교차로', 
             '271. 샛강생태공원방문자센터 앞', '297.국회3문', '298.KBS 연구동 건너편', '299.여의도 순복음교회', '4563. 여의도 은하아파트', '4564. 63스퀘어',
             '4579. 국회의사당 본관', '4580. 국회3문', '4581. 국회소통관', '4582. 국회도서관', '4583. 의정관', '4584. 국회2문', '4585. 국회6문', '4589. KRX한국거래소(1)',
             '5853. 여의도역2번출구 앞', '5854. 신한금융투자 앞', '5855. 하이투자증권 앞', '5859. 여의도 삼부아파트'],
    
    'Latitude': [
                37.52841568, 37.53123856, 37.52881622, 37.5280571, 37.52816391, 37.52626419, 37.52466583, 37.52715683, 
                37.52461243, 37.52190781, 37.5230217, 37.52483749, 37.52207947, 37.52069473, 37.5193634, 37.52267456, 
                37.51943588, 37.51913071, 37.5200882, 37.51758575, 37.51776505, 37.53105545, 37.52434921, 37.53365707, 
                37.52234268, 37.51896286, 37.52863693, 37.52750778,37.53179169, 37.51850891, 37.52070999, 37.5325737, 
                37.52974701, 37.53050232, 37.53054047, 37.53214645,37.52864075, 37.53271866, 37.52347946, 37.52260971, 
                37.52508926, 37.52464294, 37.52539444],
    
    'Longitude': [
                126.9139175, 126.9213333, 126.92453, 126.9187012, 126.9170227, 126.9205093, 126.9180222, 126.9319, 
                126.9278336, 126.9189529, 126.9208374, 126.934906, 126.9303665, 126.9258347, 126.9260483, 126.9377899, 
                126.9375992, 126.9333725, 126.9323654, 126.9296188, 126.9284134, 126.9242096, 126.9361191, 126.9121628, 126.9271011, 126.9216156, 126.912674, 126.914299, 
                126.9224548, 126.9344864, 126.9396591, 126.9140472, 126.9131699, 126.9128952, 126.9181747, 
                126.9174728, 126.9151077, 126.9179916, 126.9271164, 126.9230118, 126.9240799, 126.9234238, 126.9304581]

})

# 데이터 프레임을 GeoDataFrame으로 변환
geometry = [Point(xy) for xy in zip(data['Longitude'], data['Latitude'])]
gdf = gpd.GeoDataFrame(data, geometry=geometry)

# 여의도의 중심 좌표를 사용하여 지도 초기화
center = [37.52158, 126.9243]

# Additional data
additional_data = pd.DataFrame({
    'City': ['여의도', '여의나루', '국회의사당', '샛강'],
    'Latitude': [37.521578, 37.527145, 37.528105, 37.517274],
    'Longitude': [126.924318, 126.932807, 126.917874, 126.928422]
})

# New data
new_data = pd.DataFrame({
    'City': ['국민일보 앞', '여의나루역 1번출구 앞', '신한금융투자 앞'],
    'Latitude': [37.52881622, 37.52715683, 37.52508926],
    'Longitude': [126.92453, 126.9319, 126.9240799]
})


# 데이터 생성
by_data = pd.DataFrame({
    'X': [126.9139175, 126.9213333, 126.92453, 126.9187012, 126.9170227, 126.9205093, 126.9180222, 126.9319, 126.9278336, 126.9189529, 126.9208374, 126.934906, 126.9303665, 126.9258347, 126.9260483, 126.9377899, 126.9375992, 126.9333725, 126.9323654, 126.9296188, 126.9284134, 126.9242096, 126.9361191, 126.9121628, 126.9271011, 126.9216156, 126.912674, 126.914299, 126.9224548, 126.9344864, 126.9396591, 126.9140472, 126.9131699, 126.9128952, 126.9181747, 126.9174728, 126.9151077, 126.9179916, 126.9271164, 126.9230118, 126.9240799, 126.9234238, 126.9304581],
    'Y': [37.52841568, 37.53123856, 37.52881622, 37.5280571, 37.52816391, 37.52626419, 37.52466583, 37.52715683, 37.52461243, 37.52190781, 37.5230217, 37.52483749, 37.52207947, 37.52069473, 37.5193634, 37.52267456, 37.51943588, 37.51913071, 37.5200882, 37.51758575, 37.51776505, 37.53105545, 37.52434921, 37.53365707, 37.52234268, 37.51896286, 37.52863693, 37.52750778, 37.53179169, 37.51850891, 37.52070999, 37.5325737, 37.52974701, 37.53050232, 37.53054047, 37.53214645, 37.52864075, 37.53271866, 37.52347946, 37.52260971, 37.52508926, 37.52464294, 37.52539444],
    '이용합계': [162, 1763, 5103, 2862, 1368, 2226, 3362, 9469, 3534, 3159, 2018, 1426, 4156, 3192, 2035, 1828, 4318, 1426, 2591, 2510, 3019, 1217, 2614, 669, 2402, 633, 239, 776, 372, 996, 3031, 606, 663, 416, 583, 386, 225, 317, 1011, 1697, 5638, 1810, 1980],
    '대여건수': [89, 883, 2550, 1434, 687, 1112, 1805, 4947, 1691, 1825, 1046, 696, 2075, 1593, 1080, 927, 2059, 768, 1305, 1360, 1655, 609, 1269, 346, 1207, 380, 122, 488, 185, 506, 1469, 354, 331, 208, 324, 227, 132, 163, 507, 858, 2789, 903, 985],
    '반납건수': [73, 880, 2553, 1428, 681, 1114, 1557, 4522, 1843, 1334, 972, 730, 2081, 1599, 955, 901, 2259, 658, 1286, 1150, 1364, 608, 1345, 323, 1195, 253, 117, 288, 187, 490, 1562, 252, 332, 208, 259, 159, 93, 154, 504, 839, 2849, 907, 995]
})



# 지오데이터프레임 생성
geometry_2 = [Point(xy) for xy in zip(df['X'], df['Y'])]
gdf_2 = gpd.GeoDataFrame(df, geometry=geometry)




# Plotly Express를 사용하여 지도 플로팅
fig = px.scatter_mapbox(
    df,
    lat='Y',
    lon='X',
    size='이용합계',
    color='대여건수',
    size_max=40,
    zoom=12,
    mapbox_style="carto-positron"
)

app = dash.Dash(__name__)

# 대시보드의 탭 구성
app.layout = html.Div([
    dcc.Tabs([
        dcc.Tab(label='Plotly Express Map', children=[
            dcc.Graph(figure=fig)
        ]),
        dcc.Tab(label='Folium Map', children=[
            html.Iframe(srcDoc=io.open(html_file_path, 'r', encoding='utf-8').read(), width='100%', height='500')
        ])
    ])
])

# 앱 실행
if __name__ == '__main__':
    app.run_server(debug=True, port=1015)


In [ ]:
# Dash 앱 초기화
app = dash.Dash(__name__)

# Dash 레이아웃 구성
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

# 앱 실행
if __name__ == '__main__':
    app.run_server(debug=True, port=1012)

# Combine all the data
combined_data = pd.concat([data, additional_data, new_data], ignore_index=True)

# Convert the combined data to GeoDataFrame
combined_geometry = [Point(xy) for xy in zip(combined_data['Longitude'], combined_data['Latitude'])]
combined_gdf = gpd.GeoDataFrame(combined_data, geometry=combined_geometry)

# Update the map with the new GeoDataFrame
m = folium.Map(location=center, zoom_start=15)

# Add markers for all locations
for idx, row in combined_gdf.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['City']).add_to(m)

# Add red buffer for each location in the original data
buffer_distance_meters_red = 0.0015  # Set the buffer distance to 0.0015 meters (red)
gdf['Buffer'] = gdf['geometry'].buffer(buffer_distance_meters_red)

for idx, row in gdf.iterrows():
    folium.GeoJson(row['Buffer'].__geo_interface__,
                   style_function=lambda x: {'fillColor': 'red'}).add_to(m)

# Add blue buffer for each location in the new and additional data
buffer_distance_meters_blue = 300  # Set the buffer distance to 300 meters (blue)
for idx, row in pd.concat([additional_data, new_data], ignore_index=True).iterrows():
    folium.Circle([row['Latitude'], row['Longitude']], radius=buffer_distance_meters_blue, color='blue', fill=True).add_to(m)

# Add green buffer for each location in the new data
buffer_distance_meters_green = 0.002  # Set the buffer distance to 0.002 meters (green)
for idx, row in new_data.iterrows():
    point = Point(row['Longitude'], row['Latitude'])
    buffer_geometry = point.buffer(buffer_distance_meters_green)
    folium.GeoJson(buffer_geometry.__geo_interface__,
                   style_function=lambda x: {'fillColor': 'green'}).add_to(m)

# Save the map as an HTML file
html_file_path = 'map_with_red_blue_green_buffers.html'
m.save(html_file_path)

# Dash 앱 초기화
app = dash.Dash(__name__)

# Dash 레이아웃 구성
app.layout = html.Div([
    html.Iframe(srcDoc=io.open(html_file_path, 'r', encoding='utf-8').read(), width='100%', height='500')
])

# 앱 실행
if __name__ == '__main__':
    app.run_server(debug=True, port=1014)

In [13]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# 데이터 프레임 생성
data = pd.DataFrame({
    'X': [126.9139175, 126.9213333, 126.92453, 126.9187012, 126.9170227, 126.9205093, 126.9180222, 126.9319, 126.9278336, 126.9189529, 126.9208374, 126.934906, 126.9303665, 126.9258347, 126.9260483, 126.9377899, 126.9375992, 126.9333725, 126.9323654, 126.9296188, 126.9284134, 126.9242096, 126.9361191, 126.9121628, 126.9271011, 126.9216156, 126.912674, 126.914299, 126.9224548, 126.9344864, 126.9396591, 126.9140472, 126.9131699, 126.9128952, 126.9181747, 126.9174728, 126.9151077, 126.9179916, 126.9271164, 126.9230118, 126.9240799, 126.9234238, 126.9304581],
    'Y': [37.52841568, 37.53123856, 37.52881622, 37.5280571, 37.52816391, 37.52626419, 37.52466583, 37.52715683, 37.52461243, 37.52190781, 37.5230217, 37.52483749, 37.52207947, 37.52069473, 37.5193634, 37.52267456, 37.51943588, 37.51913071, 37.5200882, 37.51758575, 37.51776505, 37.53105545, 37.52434921, 37.53365707, 37.52234268, 37.51896286, 37.52863693, 37.52750778, 37.53179169, 37.51850891, 37.52070999, 37.5325737, 37.52974701, 37.53050232, 37.53054047, 37.53214645, 37.52864075, 37.53271866, 37.52347946, 37.52260971, 37.52508926, 37.52464294, 37.52539444],
    '이용합계': [162, 1763, 5103, 2862, 1368, 2226, 3362, 9469, 3534, 3159, 2018, 1426, 4156, 3192, 2035, 1828, 4318, 1426, 2591, 2510, 3019, 1217, 2614, 669, 2402, 633, 239, 776, 372, 996, 3031, 606, 663, 416, 583, 386, 225, 317, 1011, 1697, 5638, 1810, 1980],
    '대여건수': [89, 883, 2550, 1434, 687, 1112, 1805, 4947, 1691, 1825, 1046, 696, 2075, 1593, 1080, 927, 2059, 768, 1305, 1360, 1655, 609, 1269, 346, 1207, 380, 122, 488, 185, 506, 1469, 354, 331, 208, 324, 227, 132, 163, 507, 858, 2789, 903, 985],
    '반납건수': [73, 880, 2553, 1428, 681, 1114, 1557, 4522, 1843, 1334, 972, 730, 2081, 1599, 955, 901, 2259, 658, 1286, 1150, 1364, 608, 1345, 323, 1195, 253, 117, 288, 187, 490, 1562, 252, 332, 208, 259, 159, 93, 154, 504, 839, 2849, 907, 995]
})

# 지도 탭
map_tab = dcc.Tab(label='지도', children=[
    dcc.Graph(
        figure=px.scatter_mapbox(
            data,
            lat='Y',
            lon='X',
            size='반납건수',
            color='반납건수',
            size_max=40,
            zoom=12,
            mapbox_style="carto-positron"
        )
    )
])

# 산점도 탭
# scatter_tab = dcc.Tab(label='산점도', children=[
 #    dcc.Graph(
  #       figure=px.scatter(
    #         data,
      #       x='이용합계',
        #     y='대여건수',
          #   size='반납건수',
          #   color='반납건수',
       #      size_max=40
      #   )
   #  )
# ])

# 대시보드 앱 초기화
app = dash.Dash(__name__)

# 대시보드 레이아웃 구성
app.layout = html.Div([
    dcc.Tabs([
        map_tab,
        scatter_tab
    ])
])

# 앱 실행
if __name__ == '__main__':
    app.run_server(debug=True, port=1006)


In [24]:
import geopandas as gpd
from shapely.geometry import Point
import folium
from IPython.display import IFrame
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import io
import plotly.express as px
import pandas as pd

# 데이터 프레임 생성
data = pd.DataFrame({
    'City': ['200. 국회의원회관', '201. 진미파라곤 앞', '202. 국민일보 앞', '203. 국회의사당역 3번출구 옆', '204. 국회의사당역 5번출구 옆', '205. 산업은행 앞', 
             '206. KBS 앞', '207. 여의나루역 1번출구 앞', '209. 유진투자증권빌딩 앞', '213. KT앞', '214. 금융감독원 앞', '215. 여의도고교 앞', '217. NH농협은행 앞', 
             '219. 롯데캐슬엠파이어 옆', '220. 미성아파트 A동 앞', '221. 여의도초교 앞', '222. 시범아파트버스정류장 옆', '223. 진주아파트상가 앞', '224. 롯데캐슬 앞', 
             '225. 앙카라공원 앞', '226. 샛강역 1번출구 앞', '248. 초원아파트 앞', '249. 여의도중학교 옆', '260. 여의도 마리나선착장 앞', '270. 증권거래소후문교차로', 
             '271. 샛강생태공원방문자센터 앞', '297.국회3문', '298.KBS 연구동 건너편', '299.여의도 순복음교회', '4563. 여의도 은하아파트', '4564. 63스퀘어',
             '4579. 국회의사당 본관', '4580. 국회3문', '4581. 국회소통관', '4582. 국회도서관', '4583. 의정관', '4584. 국회2문', '4585. 국회6문', '4589. KRX한국거래소(1)',
             '5853. 여의도역2번출구 앞', '5854. 신한금융투자 앞', '5855. 하이투자증권 앞', '5859. 여의도 삼부아파트'],
    
    'Latitude': [
                37.52841568, 37.53123856, 37.52881622, 37.5280571, 37.52816391, 37.52626419, 37.52466583, 37.52715683, 
                37.52461243, 37.52190781, 37.5230217, 37.52483749, 37.52207947, 37.52069473, 37.5193634, 37.52267456, 
                37.51943588, 37.51913071, 37.5200882, 37.51758575, 37.51776505, 37.53105545, 37.52434921, 37.53365707, 
                37.52234268, 37.51896286, 37.52863693, 37.52750778,37.53179169, 37.51850891, 37.52070999, 37.5325737, 
                37.52974701, 37.53050232, 37.53054047, 37.53214645,37.52864075, 37.53271866, 37.52347946, 37.52260971, 
                37.52508926, 37.52464294, 37.52539444],
    
    'Longitude': [
                126.9139175, 126.9213333, 126.92453, 126.9187012, 126.9170227, 126.9205093, 126.9180222, 126.9319, 
                126.9278336, 126.9189529, 126.9208374, 126.934906, 126.9303665, 126.9258347, 126.9260483, 126.9377899, 
                126.9375992, 126.9333725, 126.9323654, 126.9296188, 126.9284134, 126.9242096, 126.9361191, 126.9121628, 126.9271011, 126.9216156, 126.912674, 126.914299, 
                126.9224548, 126.9344864, 126.9396591, 126.9140472, 126.9131699, 126.9128952, 126.9181747, 
                126.9174728, 126.9151077, 126.9179916, 126.9271164, 126.9230118, 126.9240799, 126.9234238, 126.9304581]

})

# 데이터 프레임을 GeoDataFrame으로 변환
geometry = [Point(xy) for xy in zip(data['Longitude'], data['Latitude'])]
gdf = gpd.GeoDataFrame(data, geometry=geometry)

# 여의도의 중심 좌표를 사용하여 지도 초기화
center = [37.52158, 126.9243]

# Additional data
additional_data = pd.DataFrame({
    'City': ['여의도', '여의나루', '국회의사당', '샛강'],
    'Latitude': [37.521578, 37.527145, 37.528105, 37.517274],
    'Longitude': [126.924318, 126.932807, 126.917874, 126.928422]
})

# New data
new_data = pd.DataFrame({
    'City': ['국민일보 앞', '여의나루역 1번출구 앞', '신한금융투자 앞'],
    'Latitude': [37.52881622, 37.52715683, 37.52508926],
    'Longitude': [126.92453, 126.9319, 126.9240799]
})

# Combine all the data
combined_data = pd.concat([data, additional_data, new_data], ignore_index=True)

# Convert the combined data to GeoDataFrame
combined_geometry = [Point(xy) for xy in zip(combined_data['Longitude'], combined_data['Latitude'])]
combined_gdf = gpd.GeoDataFrame(combined_data, geometry=combined_geometry)

# Update the map with the new GeoDataFrame
m = folium.Map(location=center, zoom_start=15)

# Add markers for all locations
for idx, row in combined_gdf.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['City']).add_to(m)

# Add red buffer for each location in the original data
buffer_distance_meters_red = 0.0015  # Set the buffer distance to 0.0015 meters (red)
gdf['Buffer'] = gdf['geometry'].buffer(buffer_distance_meters_red)

for idx, row in gdf.iterrows():
    folium.GeoJson(row['Buffer'].__geo_interface__,
                   style_function=lambda x: {'fillColor': 'red'}).add_to(m)

# Add blue buffer for each location in the new and additional data
buffer_distance_meters_blue = 300  # Set the buffer distance to 300 meters (blue)
for idx, row in pd.concat([additional_data, new_data], ignore_index=True).iterrows():
    folium.Circle([row['Latitude'], row['Longitude']], radius=buffer_distance_meters_blue, color='blue', fill=True).add_to(m)

# Add green buffer for each location in the new data
buffer_distance_meters_green = 0.002  # Set the buffer distance to 0.002 meters (green)
for idx, row in new_data.iterrows():
    point = Point(row['Longitude'], row['Latitude'])
    buffer_geometry = point.buffer(buffer_distance_meters_green)
    folium.GeoJson(buffer_geometry.__geo_interface__,
                   style_function=lambda x: {'fillColor': 'green'}).add_to(m)

# Save the map as an HTML file
html_file_path = 'map_with_red_blue_green_buffers.html'
m.save(html_file_path)

# Dash 앱 초기화
app = dash.Dash(__name__)

# Dash 레이아웃 구성
app.layout = html.Div([
    html.Iframe(srcDoc=io.open(html_file_path, 'r', encoding='utf-8').read(), width='100%', height='500')
])

# 앱 실행
if __name__ == '__main__':
    app.run_server(debug=True, port=1003)
    
    
    

# 데이터 프레임 생성
data = pd.DataFrame({
    'X1': [126.9139175, 126.9213333, 126.92453, 126.9187012, 126.9170227, 126.9205093, 126.9180222, 126.9319, 126.9278336, 126.9189529, 126.9208374, 126.934906, 126.9303665, 126.9258347, 126.9260483, 126.9377899, 126.9375992, 126.9333725, 126.9323654, 126.9296188, 126.9284134, 126.9242096, 126.9361191, 126.9121628, 126.9271011, 126.9216156, 126.912674, 126.914299, 126.9224548, 126.9344864, 126.9396591, 126.9140472, 126.9131699, 126.9128952, 126.9181747, 126.9174728, 126.9151077, 126.9179916, 126.9271164, 126.9230118, 126.9240799, 126.9234238, 126.9304581],
    'Y1': [37.52841568, 37.53123856, 37.52881622, 37.5280571, 37.52816391, 37.52626419, 37.52466583, 37.52715683, 37.52461243, 37.52190781, 37.5230217, 37.52483749, 37.52207947, 37.52069473, 37.5193634, 37.52267456, 37.51943588, 37.51913071, 37.5200882, 37.51758575, 37.51776505, 37.53105545, 37.52434921, 37.53365707, 37.52234268, 37.51896286, 37.52863693, 37.52750778, 37.53179169, 37.51850891, 37.52070999, 37.5325737, 37.52974701, 37.53050232, 37.53054047, 37.53214645, 37.52864075, 37.53271866, 37.52347946, 37.52260971, 37.52508926, 37.52464294, 37.52539444],
    '이용합계1': [162, 1763, 5103, 2862, 1368, 2226, 3362, 9469, 3534, 3159, 2018, 1426, 4156, 3192, 2035, 1828, 4318, 1426, 2591, 2510, 3019, 1217, 2614, 669, 2402, 633, 239, 776, 372, 996, 3031, 606, 663, 416, 583, 386, 225, 317, 1011, 1697, 5638, 1810, 1980],
    '대여건수1': [89, 883, 2550, 1434, 687, 1112, 1805, 4947, 1691, 1825, 1046, 696, 2075, 1593, 1080, 927, 2059, 768, 1305, 1360, 1655, 609, 1269, 346, 1207, 380, 122, 488, 185, 506, 1469, 354, 331, 208, 324, 227, 132, 163, 507, 858, 2789, 903, 985],
    '반납건수1': [73, 880, 2553, 1428, 681, 1114, 1557, 4522, 1843, 1334, 972, 730, 2081, 1599, 955, 901, 2259, 658, 1286, 1150, 1364, 608, 1345, 323, 1195, 253, 117, 288, 187, 490, 1562, 252, 332, 208, 259, 159, 93, 154, 504, 839, 2849, 907, 995]
})

# 지도 탭
map_tab = dcc.Tab(label='지도', children=[
    dcc.Graph(
        figure=px.scatter_mapbox(
            data,
            lat='Y1',
            lon='X1',
            size='반납건수1',
            color='반납건수1',
            size_max=40,
            zoom=12,
            mapbox_style="carto-positron"
        )
    )
])



# 대시보드 앱 초기화
app = dash.Dash(__name__)

# 대시보드 레이아웃 구성
app.layout = html.Div([
    dcc.Tabs([
        map_tab,
     #   scatter_tab,

      
        
    ])
])

# 앱 실행
if __name__ == '__main__':
    app.run_server(debug=True, port=1006)


In [37]:
import geopandas as gpd
from shapely.geometry import Point
import folium
from IPython.display import IFrame
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import io
import plotly.express as px

# 데이터 프레임 생성
data = pd.DataFrame({
    'City': ['200. 국회의원회관', '201. 진미파라곤 앞', '202. 국민일보 앞', '203. 국회의사당역 3번출구 옆', '204. 국회의사당역 5번출구 옆', '205. 산업은행 앞', 
             '206. KBS 앞', '207. 여의나루역 1번출구 앞', '209. 유진투자증권빌딩 앞', '213. KT앞', '214. 금융감독원 앞', '215. 여의도고교 앞', '217. NH농협은행 앞', 
             '219. 롯데캐슬엠파이어 옆', '220. 미성아파트 A동 앞', '221. 여의도초교 앞', '222. 시범아파트버스정류장 옆', '223. 진주아파트상가 앞', '224. 롯데캐슬 앞', 
             '225. 앙카라공원 앞', '226. 샛강역 1번출구 앞', '248. 초원아파트 앞', '249. 여의도중학교 옆', '260. 여의도 마리나선착장 앞', '270. 증권거래소후문교차로', 
             '271. 샛강생태공원방문자센터 앞', '297.국회3문', '298.KBS 연구동 건너편', '299.여의도 순복음교회', '4563. 여의도 은하아파트', '4564. 63스퀘어',
             '4579. 국회의사당 본관', '4580. 국회3문', '4581. 국회소통관', '4582. 국회도서관', '4583. 의정관', '4584. 국회2문', '4585. 국회6문', '4589. KRX한국거래소(1)',
             '5853. 여의도역2번출구 앞', '5854. 신한금융투자 앞', '5855. 하이투자증권 앞', '5859. 여의도 삼부아파트'],
    
    'Latitude': [
                37.52841568, 37.53123856, 37.52881622, 37.5280571, 37.52816391, 37.52626419, 37.52466583, 37.52715683, 
                37.52461243, 37.52190781, 37.5230217, 37.52483749, 37.52207947, 37.52069473, 37.5193634, 37.52267456, 
                37.51943588, 37.51913071, 37.5200882, 37.51758575, 37.51776505, 37.53105545, 37.52434921, 37.53365707, 
                37.52234268, 37.51896286, 37.52863693, 37.52750778,37.53179169, 37.51850891, 37.52070999, 37.5325737, 
                37.52974701, 37.53050232, 37.53054047, 37.53214645,37.52864075, 37.53271866, 37.52347946, 37.52260971, 
                37.52508926, 37.52464294, 37.52539444],
    
    'Longitude': [
                126.9139175, 126.9213333, 126.92453, 126.9187012, 126.9170227, 126.9205093, 126.9180222, 126.9319, 
                126.9278336, 126.9189529, 126.9208374, 126.934906, 126.9303665, 126.9258347, 126.9260483, 126.9377899, 
                126.9375992, 126.9333725, 126.9323654, 126.9296188, 126.9284134, 126.9242096, 126.9361191, 126.9121628, 126.9271011, 126.9216156, 126.912674, 126.914299, 
                126.9224548, 126.9344864, 126.9396591, 126.9140472, 126.9131699, 126.9128952, 126.9181747, 
                126.9174728, 126.9151077, 126.9179916, 126.9271164, 126.9230118, 126.9240799, 126.9234238, 126.9304581]
    

})



# 데이터 프레임을 GeoDataFrame으로 변환
geometry = [Point(xy) for xy in zip(data['Longitude'], data['Latitude'])]
gdf = gpd.GeoDataFrame(data, geometry=geometry)

# 여의도의 중심 좌표를 사용하여 지도 초기화
center = [37.52158, 126.9243]

# Additional data
additional_data = pd.DataFrame({
    'City': ['여의도', '여의나루', '국회의사당', '샛강'],
    'Latitude': [37.521578, 37.527145, 37.528105, 37.517274],
    'Longitude': [126.924318, 126.932807, 126.917874, 126.928422]
})

# New data
new_data = pd.DataFrame({
    'City': ['국민일보 앞', '여의나루역 1번출구 앞', '신한금융투자 앞'],
    'Latitude': [37.52881622, 37.52715683, 37.52508926],
    'Longitude': [126.92453, 126.9319, 126.9240799]
})


data_2 = pd.DataFrame({

    'X': [126.9139175, 126.9213333, 126.92453, 126.9187012, 126.9170227, 126.9205093, 126.9180222, 126.9319, 126.9278336, 126.9189529, 126.9208374, 126.934906, 126.9303665, 126.9258347, 126.9260483, 126.9377899, 126.9375992, 126.9333725, 126.9323654, 126.9296188, 126.9284134, 126.9242096, 126.9361191, 126.9121628, 126.9271011, 126.9216156, 126.912674, 126.914299, 126.9224548, 126.9344864, 126.9396591, 126.9140472, 126.9131699, 126.9128952, 126.9181747, 126.9174728, 126.9151077, 126.9179916, 126.9271164, 126.9230118, 126.9240799, 126.9234238, 126.9304581],

    'Y': [37.52841568, 37.53123856, 37.52881622, 37.5280571, 37.52816391, 37.52626419, 37.52466583, 37.52715683, 37.52461243, 37.52190781, 37.5230217, 37.52483749, 37.52207947, 37.52069473, 37.5193634, 37.52267456, 37.51943588, 37.51913071, 37.5200882, 37.51758575, 37.51776505, 37.53105545, 37.52434921, 37.53365707, 37.52234268, 37.51896286, 37.52863693, 37.52750778, 37.53179169, 37.51850891, 37.52070999, 37.5325737, 37.52974701, 37.53050232, 37.53054047, 37.53214645, 37.52864075, 37.53271866, 37.52347946, 37.52260971, 37.52508926, 37.52464294, 37.52539444],

    '이용합계': [162, 1763, 5103, 2862, 1368, 2226, 3362, 9469, 3534, 3159, 2018, 1426, 4156, 3192, 2035, 1828, 4318, 1426, 2591, 2510, 3019, 1217, 2614, 669, 2402, 633, 239, 776, 372, 996, 3031, 606, 663, 416, 583, 386, 225, 317, 1011, 1697, 5638, 1810, 1980],

    '대여건수': [89, 883, 2550, 1434, 687, 1112, 1805, 4947, 1691, 1825, 1046, 696, 2075, 1593, 1080, 927, 2059, 768, 1305, 1360, 1655, 609, 1269, 346, 1207, 380, 122, 488, 185, 506, 1469, 354, 331, 208, 324, 227, 132, 163, 507, 858, 2789, 903, 985],

    '반납건수': [73, 880, 2553, 1428, 681, 1114, 1557, 4522, 1843, 1334, 972, 730, 2081, 1599, 955, 901, 2259, 658, 1286, 1150, 1364, 608, 1345, 323, 1195, 253, 117, 288, 187, 490, 1562, 252, 332, 208, 259, 159, 93, 154, 504, 839, 2849, 907, 995]

    
    
})


# Combine all the data
combined_data = pd.concat([data, additional_data, new_data], ignore_index=True)

# Convert the combined data to GeoDataFrame
combined_geometry = [Point(xy) for xy in zip(combined_data['Longitude'], combined_data['Latitude'])]
combined_gdf = gpd.GeoDataFrame(combined_data, geometry=combined_geometry)

# Update the map with the new GeoDataFrame
m = folium.Map(location=center, zoom_start=15)

# Add markers for all locations
for idx, row in combined_gdf.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['City']).add_to(m)

# Add red buffer for each location in the original data
buffer_distance_meters_red = 0.0015  # Set the buffer distance to 0.0015 meters (red)
gdf['Buffer'] = gdf['geometry'].buffer(buffer_distance_meters_red)

for idx, row in gdf.iterrows():
    folium.GeoJson(row['Buffer'].__geo_interface__,
                   style_function=lambda x: {'fillColor': 'red'}).add_to(m)

# Add blue buffer for each location in the new and additional data
buffer_distance_meters_blue = 300  # Set the buffer distance to 300 meters (blue)
for idx, row in pd.concat([additional_data, new_data], ignore_index=True).iterrows():
    folium.Circle([row['Latitude'], row['Longitude']], radius=buffer_distance_meters_blue, color='blue', fill=True).add_to(m)

# Add green buffer for each location in the new data
buffer_distance_meters_green = 0.002  # Set the buffer distance to 0.002 meters (green)
for idx, row in new_data.iterrows():
    point = Point(row['Longitude'], row['Latitude'])
    buffer_geometry = point.buffer(buffer_distance_meters_green)
    folium.GeoJson(buffer_geometry.__geo_interface__,
                   style_function=lambda x: {'fillColor': 'green'}).add_to(m)

# Save the map as an HTML file
html_file_path = 'map_with_red_blue_green_buffers.html'
m.save(html_file_path)





# Plotly 지도를 포함한 첫 번째 탭
map_tab = dcc.Tab(label='지도', children=[
    html.Iframe(srcDoc=io.open(html_file_path, 'r', encoding='utf-8').read(), width='100%', height='500')
])

# 산점도를 포함한 두 번째 탭
scatter_tab = dcc.Tab(label='산점도', children=[
    dcc.Graph(
        figure=px.scatter_mapbox(
            combined_data,
            lat='Latitude',
            lon='Longitude',
            size=40,
            color='orange',
            size_max=40,
            zoom=12,
            mapbox_style="carto-positron"
        )
    )
])

# 대시보드 앱 초기화
app = dash.Dash(__name__)

# 대시보드 레이아웃 구성
app.layout = html.Div([
    dcc.Tabs([
        map_tab,
        scatter_tab,
    ])
])

# 앱 실행
if __name__ == '__main__':
    app.run_server(debug=True, port=1006)


ValueError: Value of 'size' is not the name of a column in 'data_frame'. Expected one of ['City', 'Latitude', 'Longitude'] but received: 40

In [38]:
import geopandas as gpd
from shapely.geometry import Point
import folium
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Existing DataFrame creation
data = pd.DataFrame({
    'City': ['200. 국회의원회관', '201. 진미파라곤 앞', '202. 국민일보 앞', '203. 국회의사당역 3번출구 옆', '204. 국회의사당역 5번출구 옆', '205. 산업은행 앞', 
             '206. KBS 앞', '207. 여의나루역 1번출구 앞', '209. 유진투자증권빌딩 앞', '213. KT앞', '214. 금융감독원 앞', '215. 여의도고교 앞', '217. NH농협은행 앞', 
             '219. 롯데캐슬엠파이어 옆', '220. 미성아파트 A동 앞', '221. 여의도초교 앞', '222. 시범아파트버스정류장 옆', '223. 진주아파트상가 앞', '224. 롯데캐슬 앞', 
             '225. 앙카라공원 앞', '226. 샛강역 1번출구 앞', '248. 초원아파트 앞', '249. 여의도중학교 옆', '260. 여의도 마리나선착장 앞', '270. 증권거래소후문교차로', 
             '271. 샛강생태공원방문자센터 앞', '297.국회3문', '298.KBS 연구동 건너편', '299.여의도 순복음교회', '4563. 여의도 은하아파트', '4564. 63스퀘어',
             '4579. 국회의사당 본관', '4580. 국회3문', '4581. 국회소통관', '4582. 국회도서관', '4583. 의정관', '4584. 국회2문', '4585. 국회6문', '4589. KRX한국거래소(1)',
             '5853. 여의도역2번출구 앞', '5854. 신한금융투자 앞', '5855. 하이투자증권 앞', '5859. 여의도 삼부아파트'],
    
    'Latitude': [
                37.52841568, 37.53123856, 37.52881622, 37.5280571, 37.52816391, 37.52626419, 37.52466583, 37.52715683, 
                37.52461243, 37.52190781, 37.5230217, 37.52483749, 37.52207947, 37.52069473, 37.5193634, 37.52267456, 
                37.51943588, 37.51913071, 37.5200882, 37.51758575, 37.51776505, 37.53105545, 37.52434921, 37.53365707, 
                37.52234268, 37.51896286, 37.52863693, 37.52750778,37.53179169, 37.51850891, 37.52070999, 37.5325737, 
                37.52974701, 37.53050232, 37.53054047, 37.53214645,37.52864075, 37.53271866, 37.52347946, 37.52260971, 
                37.52508926, 37.52464294, 37.52539444],
    
    'Longitude': [
                126.9139175, 126.9213333, 126.92453, 126.9187012, 126.9170227, 126.9205093, 126.9180222, 126.9319, 
                126.9278336, 126.9189529, 126.9208374, 126.934906, 126.9303665, 126.9258347, 126.9260483, 126.9377899, 
                126.9375992, 126.9333725, 126.9323654, 126.9296188, 126.9284134, 126.9242096, 126.9361191, 126.9121628, 126.9271011, 126.9216156, 126.912674, 126.914299, 
                126.9224548, 126.9344864, 126.9396591, 126.9140472, 126.9131699, 126.9128952, 126.9181747, 
                126.9174728, 126.9151077, 126.9179916, 126.9271164, 126.9230118, 126.9240799, 126.9234238, 126.9304581]
})

# Data for additional and new locations
additional_data = pd.DataFrame({
    'City': ['여의도', '여의나루', '국회의사당', '샛강'],
    'Latitude': [37.521578, 37.527145, 37.528105, 37.517274],
    'Longitude': [126.924318, 126.932807, 126.917874, 126.928422]
})

new_data = pd.DataFrame({
    'City': ['국민일보 앞', '여의나루역 1번출구 앞', '신한금융투자 앞'],
    'Latitude': [37.52881622, 37.52715683, 37.52508926],
    'Longitude': [126.92453, 126.9319, 126.9240799]
})

# Combine all the data
combined_data = pd.concat([data, additional_data, new_data], ignore_index=True)

# Convert the combined data to GeoDataFrame
combined_geometry = [Point(xy) for xy in zip(combined_data['Longitude'], combined_data['Latitude'])]
combined_gdf = gpd.GeoDataFrame(combined_data, geometry=combined_geometry)

# 여의도의 중심 좌표를 사용하여 지도 초기화
center = [37.52158, 126.9243]

# Update the map with the new GeoDataFrame
m = folium.Map(location=center, zoom_start=15)

# Add markers for all locations
for idx, row in combined_gdf.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['City']).add_to(m)

# Add red buffer for each location in the original data
buffer_distance_meters_red = 0.0015  # Set the buffer distance to 0.0015 meters (red)
data['Buffer'] = gdf['geometry'].buffer(buffer_distance_meters_red)

for idx, row in data.iterrows():
    folium.GeoJson(row['Buffer'].__geo_interface__,
                   style_function=lambda x: {'fillColor': 'red'}).add_to(m)

# Add blue buffer for each location in the new and additional data
buffer_distance_meters_blue = 300  # Set the buffer distance to 300 meters (blue)
for idx, row in pd.concat([additional_data, new_data], ignore_index=True).iterrows():
    folium.Circle([row['Latitude'], row['Longitude']], radius=buffer_distance_meters_blue, color='blue', fill=True).add_to(m)

# Add green buffer for each location in the new data
buffer_distance_meters_green = 0.002  # Set the buffer distance to 0.002 meters (green)
for idx, row in new_data.iterrows():
    point = Point(row['Longitude'], row['Latitude'])
    buffer_geometry = point.buffer(buffer_distance_meters_green)
    folium.GeoJson(buffer_geometry.__geo_interface__,
                   style_function=lambda x: {'fillColor': 'green'}).add_to(m)

# Save the map as an HTML file
html_file_path = 'map_with_red_blue_green_buffers.html'
m.save(html_file_path)


In [33]:
import geopandas as gpd
from shapely.geometry import Point
import folium
from IPython.display import IFrame
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import io
import plotly.express as px

# 데이터 프레임 생성
data = pd.DataFrame({
    'City': ['200. 국회의원회관', '201. 진미파라곤 앞', '202. 국민일보 앞', '203. 국회의사당역 3번출구 옆', '204. 국회의사당역 5번출구 옆', '205. 산업은행 앞', 
             '206. KBS 앞', '207. 여의나루역 1번출구 앞', '209. 유진투자증권빌딩 앞', '213. KT앞', '214. 금융감독원 앞', '215. 여의도고교 앞', '217. NH농협은행 앞', 
             '219. 롯데캐슬엠파이어 옆', '220. 미성아파트 A동 앞', '221. 여의도초교 앞', '222. 시범아파트버스정류장 옆', '223. 진주아파트상가 앞', '224. 롯데캐슬 앞', 
             '225. 앙카라공원 앞', '226. 샛강역 1번출구 앞', '248. 초원아파트 앞', '249. 여의도중학교 옆', '260. 여의도 마리나선착장 앞', '270. 증권거래소후문교차로', 
             '271. 샛강생태공원방문자센터 앞', '297.국회3문', '298.KBS 연구동 건너편', '299.여의도 순복음교회', '4563. 여의도 은하아파트', '4564. 63스퀘어',
             '4579. 국회의사당 본관', '4580. 국회3문', '4581. 국회소통관', '4582. 국회도서관', '4583. 의정관', '4584. 국회2문', '4585. 국회6문', '4589. KRX한국거래소(1)',
             '5853. 여의도역2번출구 앞', '5854. 신한금융투자 앞', '5855. 하이투자증권 앞', '5859. 여의도 삼부아파트'],
    
    'Latitude': [
                37.52841568, 37.53123856, 37.52881622, 37.5280571, 37.52816391, 37.52626419, 37.52466583, 37.52715683, 
                37.52461243, 37.52190781, 37.5230217, 37.52483749, 37.52207947, 37.52069473, 37.5193634, 37.52267456, 
                37.51943588, 37.51913071, 37.5200882, 37.51758575, 37.51776505, 37.53105545, 37.52434921, 37.53365707, 
                37.52234268, 37.51896286, 37.52863693, 37.52750778,37.53179169, 37.51850891, 37.52070999, 37.5325737, 
                37.52974701, 37.53050232, 37.53054047, 37.53214645,37.52864075, 37.53271866, 37.52347946, 37.52260971, 
                37.52508926, 37.52464294, 37.52539444],
    
    'Longitude': [
                126.9139175, 126.9213333, 126.92453, 126.9187012, 126.9170227, 126.9205093, 126.9180222, 126.9319, 
                126.9278336, 126.9189529, 126.9208374, 126.934906, 126.9303665, 126.9258347, 126.9260483, 126.9377899, 
                126.9375992, 126.9333725, 126.9323654, 126.9296188, 126.9284134, 126.9242096, 126.9361191, 126.9121628, 126.9271011, 126.9216156, 126.912674, 126.914299, 
                126.9224548, 126.9344864, 126.9396591, 126.9140472, 126.9131699, 126.9128952, 126.9181747, 
                126.9174728, 126.9151077, 126.9179916, 126.9271164, 126.9230118, 126.9240799, 126.9234238, 126.9304581],
    
    'X': [126.9139175, 126.9213333, 126.92453, 126.9187012, 126.9170227, 126.9205093, 126.9180222, 126.9319, 126.9278336, 126.9189529, 126.9208374, 126.934906, 126.9303665, 126.9258347, 126.9260483, 126.9377899, 126.9375992, 126.9333725, 126.9323654, 126.9296188, 126.9284134, 126.9242096, 126.9361191, 126.9121628, 126.9271011, 126.9216156, 126.912674, 126.914299, 126.9224548, 126.9344864, 126.9396591, 126.9140472, 126.9131699, 126.9128952, 126.9181747, 126.9174728, 126.9151077, 126.9179916, 126.9271164, 126.9230118, 126.9240799, 126.9234238, 126.9304581],

    'Y': [37.52841568, 37.53123856, 37.52881622, 37.5280571, 37.52816391, 37.52626419, 37.52466583, 37.52715683, 37.52461243, 37.52190781, 37.5230217, 37.52483749, 37.52207947, 37.52069473, 37.5193634, 37.52267456, 37.51943588, 37.51913071, 37.5200882, 37.51758575, 37.51776505, 37.53105545, 37.52434921, 37.53365707, 37.52234268, 37.51896286, 37.52863693, 37.52750778, 37.53179169, 37.51850891, 37.52070999, 37.5325737, 37.52974701, 37.53050232, 37.53054047, 37.53214645, 37.52864075, 37.53271866, 37.52347946, 37.52260971, 37.52508926, 37.52464294, 37.52539444],

    '이용합계': [162, 1763, 5103, 2862, 1368, 2226, 3362, 9469, 3534, 3159, 2018, 1426, 4156, 3192, 2035, 1828, 4318, 1426, 2591, 2510, 3019, 1217, 2614, 669, 2402, 633, 239, 776, 372, 996, 3031, 606, 663, 416, 583, 386, 225, 317, 1011, 1697, 5638, 1810, 1980],

    '대여건수': [89, 883, 2550, 1434, 687, 1112, 1805, 4947, 1691, 1825, 1046, 696, 2075, 1593, 1080, 927, 2059, 768, 1305, 1360, 1655, 609, 1269, 346, 1207, 380, 122, 488, 185, 506, 1469, 354, 331, 208, 324, 227, 132, 163, 507, 858, 2789, 903, 985],

    '반납건수': [73, 880, 2553, 1428, 681, 1114, 1557, 4522, 1843, 1334, 972, 730, 2081, 1599, 955, 901, 2259, 658, 1286, 1150, 1364, 608, 1345, 323, 1195, 253, 117, 288, 187, 490, 1562, 252, 332, 208, 259, 159, 93, 154, 504, 839, 2849, 907, 995]

    
    
})

# 데이터 프레임을 GeoDataFrame으로 변환
geometry = [Point(xy) for xy in zip(data['Longitude'], data['Latitude'])]
gdf = gpd.GeoDataFrame(data, geometry=geometry)

# 여의도의 중심 좌표를 사용하여 지도 초기화
center = [37.52158, 126.9243]

# Additional data
additional_data = pd.DataFrame({
    'City': ['여의도', '여의나루', '국회의사당', '샛강'],
    'Latitude': [37.521578, 37.527145, 37.528105, 37.517274],
    'Longitude': [126.924318, 126.932807, 126.917874, 126.928422]
})

# New data
new_data = pd.DataFrame({
    'City': ['국민일보 앞', '여의나루역 1번출구 앞', '신한금융투자 앞'],
    'Latitude': [37.52881622, 37.52715683, 37.52508926],
    'Longitude': [126.92453, 126.9319, 126.9240799]
})

# Combine all the data
combined_data = pd.concat([data, additional_data, new_data], ignore_index=True)

# Drop rows with NaN in '반납건수' column
combined_data = combined_data.dropna(subset=['반납건수'])

# Convert the combined data to GeoDataFrame
combined_geometry = [Point(xy) for xy in zip(combined_data['Longitude'], combined_data['Latitude'])]
combined_gdf = gpd.GeoDataFrame(combined_data, geometry=combined_geometry)

# Update the map with the new GeoDataFrame
m = folium.Map(location=center, zoom_start=15)

# Add markers for all locations
for idx, row in combined_gdf.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['City']).add_to(m)

# Add red buffer for each location in the original data
buffer_distance_meters_red = 0.0015  # Set the buffer distance to 0.0015 meters (red)
gdf['Buffer'] = gdf['geometry'].buffer(buffer_distance_meters_red)

for idx, row in gdf.iterrows():
    folium.GeoJson(row['Buffer'].__geo_interface__,
                   style_function=lambda x: {'fillColor': 'red'}).add_to(m)

# Add blue buffer for each location in the new and additional data
buffer_distance_meters_blue = 300  # Set the buffer distance to 300 meters (blue)
for idx, row in pd.concat([additional_data, new_data], ignore_index=True).iterrows():
    folium.Circle([row['Latitude'], row['Longitude']], radius=buffer_distance_meters_blue, color='blue', fill=True).add_to(m)

# Add green buffer for each location in the new data
buffer_distance_meters_green = 0.002  # Set the buffer distance to 0.002 meters (green)
for idx, row in new_data.iterrows():
    point = Point(row['Longitude'], row['Latitude'])
    buffer_geometry = point.buffer(buffer_distance_meters_green)
    folium.GeoJson(buffer_geometry.__geo_interface__,
                   style_function=lambda x: {'fillColor': 'green'}).add_to(m)

# Save the map as an HTML file
html_file_path = 'map_with_red_blue_green_buffers.html'
m.save(html_file_path)

# Plotly 지도를 포함한 첫 번째 탭
map_tab = dcc.Tab(label='지도', children=[
    html.Iframe(srcDoc=io.open(html_file_path, 'r', encoding='utf-8').read(), width='100%', height='500')
])

# 산점도를 포함한 두 번째 탭
scatter_tab = dcc.Tab(label='산점도', children=[
    dcc.Graph(
        figure=px.scatter_mapbox(
            combined_data,
            lat='Latitude',
            lon='Longitude',
            size='반납건수',
            color='


SyntaxError: unterminated string literal (detected at line 127) (1112670392.py, line 127)

In [31]:
import geopandas as gpd
from shapely.geometry import Point
import folium
from IPython.display import IFrame
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import io
import plotly.express as px

# 데이터 프레임 생성
data = pd.DataFrame({
    'City': ['200. 국회의원회관', '201. 진미파라곤 앞', '202. 국민일보 앞', '203. 국회의사당역 3번출구 옆', '204. 국회의사당역 5번출구 옆', '205. 산업은행 앞', 
             '206. KBS 앞', '207. 여의나루역 1번출구 앞', '209. 유진투자증권빌딩 앞', '213. KT앞', '214. 금융감독원 앞', '215. 여의도고교 앞', '217. NH농협은행 앞', 
             '219. 롯데캐슬엠파이어 옆', '220. 미성아파트 A동 앞', '221. 여의도초교 앞', '222. 시범아파트버스정류장 옆', '223. 진주아파트상가 앞', '224. 롯데캐슬 앞', 
             '225. 앙카라공원 앞', '226. 샛강역 1번출구 앞', '248. 초원아파트 앞', '249. 여의도중학교 옆', '260. 여의도 마리나선착장 앞', '270. 증권거래소후문교차로', 
             '271. 샛강생태공원방문자센터 앞', '297.국회3문', '298.KBS 연구동 건너편', '299.여의도 순복음교회', '4563. 여의도 은하아파트', '4564. 63스퀘어',
             '4579. 국회의사당 본관', '4580. 국회3문', '4581. 국회소통관', '4582. 국회도서관', '4583. 의정관', '4584. 국회2문', '4585. 국회6문', '4589. KRX한국거래소(1)',
             '5853. 여의도역2번출구 앞', '5854. 신한금융투자 앞', '5855. 하이투자증권 앞', '5859. 여의도 삼부아파트'],
    
    'Latitude': [
                37.52841568, 37.53123856, 37.52881622, 37.5280571, 37.52816391, 37.52626419, 37.52466583, 37.52715683, 
                37.52461243, 37.52190781, 37.5230217, 37.52483749, 37.52207947, 37.52069473, 37.5193634, 37.52267456, 
                37.51943588, 37.51913071, 37.5200882, 37.51758575, 37.51776505, 37.53105545, 37.52434921, 37.53365707, 
                37.52234268, 37.51896286, 37.52863693, 37.52750778,37.53179169, 37.51850891, 37.52070999, 37.5325737, 
                37.52974701, 37.53050232, 37.53054047, 37.53214645,37.52864075, 37.53271866, 37.52347946, 37.52260971, 
                37.52508926, 37.52464294, 37.52539444],
    
    'Longitude': [
                126.9139175, 126.9213333, 126.92453, 126.9187012, 126.9170227, 126.9205093, 126.9180222, 126.9319, 
                126.9278336, 126.9189529, 126.9208374, 126.934906, 126.9303665, 126.9258347, 126.9260483, 126.9377899, 
                126.9375992, 126.9333725, 126.9323654, 126.9296188, 126.9284134, 126.9242096, 126.9361191, 126.9121628, 126.9271011, 126.9216156, 126.912674, 126.914299, 
                126.9224548, 126.9344864, 126.9396591, 126.9140472, 126.9131699, 126.9128952, 126.9181747, 
                126.9174728, 126.9151077, 126.9179916, 126.9271164, 126.9230118, 126.9240799, 126.9234238, 126.9304581]
    
})

# 데이터 프레임을 GeoDataFrame으로 변환
geometry = [Point(xy) for xy in zip(data['Longitude'], data['Latitude'])]
gdf = gpd.GeoDataFrame(data, geometry=geometry)

# 여의도의 중심 좌표를 사용하여 지도 초기화
center = [37.52158, 126.9243]

# Additional data
additional_data = pd.DataFrame({
    'City': ['여의도', '여의나루', '국회의사당', '샛강'],
    'Latitude': [37.521578, 37.527145, 37.528105, 37.517274],
    'Longitude': [126.924318, 126.932807, 126.917874, 126.928422]
})

# New data
new_data = pd.DataFrame({
    'City': ['국민일보 앞', '여의나루역 1번출구 앞', '신한금융투자 앞'],
    'Latitude': [37.52881622, 37.52715683, 37.52508926],
    'Longitude': [126.92453, 126.9319, 126.9240799]
})

# Combine all the data
combined_data = pd.concat([data, additional_data, new_data], ignore_index=True)

# Convert the combined data to GeoDataFrame
combined_geometry = [Point(xy) for xy in zip(combined_data['Longitude'], combined_data['Latitude'])]
combined_gdf = gpd.GeoDataFrame(combined_data, geometry=combined_geometry)

# Update the map with the new GeoDataFrame
m = folium.Map(location=center, zoom_start=15)

# Add markers for all locations
for idx, row in combined_gdf.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['City']).add_to(m)

# Add red buffer for each location in the original data
buffer_distance_meters_red = 0.0015  # Set the buffer distance to 0.0015 meters (red)
gdf['Buffer'] = gdf['geometry'].buffer(buffer_distance_meters_red)

for idx, row in gdf.iterrows():
    folium.GeoJson(row['Buffer'].__geo_interface__,
                   style_function=lambda x: {'fillColor': 'red'}).add_to(m)

# Add blue buffer for each location in the new and additional data
buffer_distance_meters_blue = 300  # Set the buffer distance to 300 meters (blue)
for idx, row in pd.concat([additional_data, new_data], ignore_index=True).iterrows():
    folium.Circle([row['Latitude'], row['Longitude']], radius=buffer_distance_meters_blue, color='blue', fill=True).add_to(m)

# Add green buffer for each location in the new data
buffer_distance_meters_green = 0.002  # Set the buffer distance to 0.002 meters (green)
for idx, row in new_data.iterrows():
    point = Point(row['Longitude'], row['Latitude'])
    buffer_geometry = point.buffer(buffer_distance_meters_green)
    folium.GeoJson(buffer_geometry.__geo_interface__,
                   style_function=lambda x: {'fillColor': 'green'}).add_to(m)

# Save the map as an HTML file
html_file_path = 'map_with_red_blue_green_buffers.html'
m.save(html_file_path)

# Plotly 지도를 포함한 첫 번째 탭
map_tab = dcc.Tab(label='지도', children=[
    html.Iframe(srcDoc=io.open(html_file_path, 'r', encoding='utf-8').read(), width='100%', height='500')
])

# 산점도를 포함한 두 번째 탭
scatter_tab = dcc.Tab(label='산점도', children=[
    dcc.Graph(
        figure=px.scatter_mapbox(
            combined_data,
            lat='Latitude',
            lon='Longitude',
            size_max=40,
            zoom=12,
            mapbox_style="carto-positron"
        )
    )
])

# 대시보드 앱 초기화
app = dash.Dash(__name__)

# 대시보드 레이아웃 구성
app.layout = html.Div([
    dcc.Tabs([
        map_tab,
        scatter_tab,
    ])
])

# 앱 실행
if __name__ == '__main__':
    app.run_server(debug=True, port=1006)
